In [1]:
#Set up Environment

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%pylab inline
from string import ascii_letters
import sys
import re
#Preprocessing
import nltk
import os
import string

Populating the interactive namespace from numpy and matplotlib


In [2]:
import TextCleaning
import importlib

In [2]:
#Visualize data
#raw_df=pd.read_csv('/sfs/qumulo/qhome/sc2pg/src/prnd/publicrd/data/prd/DigitalOcean_Backup/public_rd/working/federal_reporter/abstracts_federal_reporter_combined.csv',engine='python')

# reading from new link on Rivanna
raw_df=pd.read_csv('~/prd/publicrd/data/prd/RND Topic Modelling/abstracts_federal_reporter_combined.csv',engine='python')

print('Columns')
print(raw_df.columns)
print('Descriptive Stats')
print(raw_df.describe())
print('Length')
print(len(raw_df))

Columns
Index(['PROJECT_ID', 'ABSTRACT', 'FY', 'FIRST_CHAR', 'LAST_CHAR', 'DEPARTMENT',
       'AGENCY', 'IC_CENTER', 'PROJECT_NUMBER', 'PROJECT_TITLE',
       'PROJECT_TERMS', 'CONTACT_PI_PROJECT_LEADER', 'OTHER_PIS',
       'ORGANIZATION_NAME', 'CFDA_CODE', 'FY_TOTAL_COST'],
      dtype='object')
Descriptive Stats
         PROJECT_ID             FY  FY_TOTAL_COST
count  5.500880e+05  550088.000000   4.256850e+05
mean   4.980384e+05    2012.247477   4.509841e+05
std    3.262050e+05       3.183765   1.727112e+06
min    1.008600e+04    2008.000000   1.000000e+00
25%    1.790538e+05    2009.000000   1.390020e+05
50%    4.880660e+05    2012.000000   2.917820e+05
75%    7.817102e+05    2015.000000   4.500000e+05
max    1.101940e+06    2018.000000   3.227983e+08
Length
550088


In [4]:
df = TextCleaning.remove_nulls(raw_df, "ABSTRACT")
df = TextCleaning.remove_duplicates(df)
df = TextCleaning.create_working_abstract_col(df)

3 nulls in  ABSTRACT . These rows removed.
11 duplicate abstracts removed
0 project ID duplicates - not removed


In [5]:
#importlib.reload(TextCleaning)
df = TextCleaning.remove_short_abstracts(df,limit=150)

1270 short abstracts removed


In [6]:
# Start phrases to remove

start_phrases=['****TECHNICAL ABSTRACT****','****Technical Abstract****',
               '****Non Technical Abstract****','*** Non- Technical Abstract ***','**Non-Technical Abstract**',
               '*****NON-TECHNICAL ABSTRACT*****','***** NON-TECHNICAL ABSTRACT *****',
               '****NONTECHNICAL ABSTRACT****','****Non-Technical Abstract****','*Non-technical Abstract*',
               '*****NON-TECHNICAL ABSTRACT*****','****NON-TECHNICAL ABSTRACT****',
               '***NON-TECHNICAL ABSTRACT***','****Nontechnical abstract****',
               'TECHNICAL SUMMARY', 'NONTECHNICAL SUMMARY','NON-TECHNICAL SUMMARY','Non-technical description',
               'DESCRIPTION (Provided by the applicant)','DESCRIPTION (provided by investigator)',  'DESCRIPTION (provided by applicant)',
               'Project Summary/Abstract','PROJECT SUMMARY/ABSTRACT',
               'ABSTRACT','abstract','Proposal Abstract','Abstract','RESEARCH ABSTRACT',
               'PROJECT SUMMARY','Project Summary','SUMMARY','RESEARCH SUMMARY',
               'Proposal',
               'DESCRIPTION','Description','PROJECT DESCRIPTION'
               'NARRATIVE',
               '(See instructions):','\t',
              'FOR CENTER APPLICATION (provided by the investigator):','Objective(s)',      'EXCEED THE SPACE PROVIDED',
               'Provided by Applicant','Provided by applicant','provided by applicant','PROVIDED BY APPLICANT',
               'Provided by Candidate','Provided by candidate','provided by candidate','PROVIDED BY CANDIDATE',
              'one page and must contain a summary of the proposed activity suitable for dissemination to thepublic. It should be a self-contained description of the project and should contain a statement of objectives and methods to be employed.It should be informative to other persons working in the same or related fields and insofar as possible understandable to a technically liter-ate lay reader. This Abstract must not include any proprietary/confidential information.* Please click the add attachment button to complete this entry.']

wa = 'working_abstract'

df[wa]=df[wa].apply(str.lstrip,args=[' ?-_^. :,!;¿|()[]]#%>﻿&\''])

#Remove found phrases
for phrase in start_phrases:
    df[wa]=df[wa].apply(TextCleaning.remove_phrase,args=[phrase,'Start']).apply(str.lstrip,args=[' :./)'])
#Repeated in case the order of project summary/abstract varies
for phrase in start_phrases:
    df[wa]=df[wa].apply(TextCleaning.remove_phrase,args=[phrase,'Start']).apply(str.lstrip,args=[' :./)'])

df[wa]=df[wa].apply(str.lstrip,args=['?-*_^. :,!;=¿|]#%>&-\t\n']) #Often, sentences will start with - or *, but they indicate other quality issues and don't end with them,so it's okay to remove them
    
df.drop(df[df[wa].apply(len)==0].index,axis=0,inplace=True)
    
# update Start Char column in df
df['Start Char']=df[wa].apply(lambda x: x[0])


In [7]:
# remove junk at end

end_phrases = ['(End of Abstract)',"End of Abstract", '(Abstract end)' "(END OF ABSTRACT)", '(End of abstract.)','(Abstract End)','(End 0f Abstract)','(End of Abstract.)','(End of Absract)',
               '(Abstract below)','(End of Reviewers\' Comment)','(End Abstract)','(End of abstract)','(End of abstract)',
               'PERFORMANCE SITE ========================================Section End===========================================',
                'KEY PERSONNEL ========================================Section End===========================================',
               '[summary truncated at 7800 characters]', 
               'This award reflects NSF\'s statutory mission and has been deemed worthy of support through evaluation using the Foundation\'s intellectual merit and broader impacts review criteria.',
               'Project Description Page 6', 'Page 1 of 1', 'Project Summary/Abstract Page 6',
               'Project Description Page 7', 'Project Summary/Abstract Page 7', 'Pag 1 o 1', 
               'Page 2 Number pages consecutively at the bottom throughout Form Page 2',
               'This award reflects NSF\'s statutory mission and has been deemed worthy of support through evaluation using the Foundation\'s intellectual merit and broader impacts review criteria.']

df = TextCleaning.remove_junk_end(df, 'working_abstract', end_phrases) 

(End of Abstract)
End of Abstract
(Abstract end)(END OF ABSTRACT)
(End of abstract.)
(Abstract End)
(End 0f Abstract)
(End of Abstract.)
(End of Absract)
(Abstract below)
(End of Reviewers' Comment)
(End Abstract)
(End of abstract)
(End of abstract)
PERFORMANCE SITE ========================================Section End===========================================
KEY PERSONNEL ========================================Section End===========================================
[summary truncated at 7800 characters]
This award reflects NSF's statutory mission and has been deemed worthy of support through evaluation using the Foundation's intellectual merit and broader impacts review criteria.
Project Description Page 6
Page 1 of 1
Project Summary/Abstract Page 6
Project Description Page 7
Project Summary/Abstract Page 7
Pag 1 o 1
Page 2 Number pages consecutively at the bottom throughout Form Page 2
This award reflects NSF's statutory mission and has been deemed worthy of support through evaluatio

In [8]:
punctuation=['!','?','.']

#y='*' #Generally if last char is '*', it comes in as '***' following a complete sentence, so unlikely to be 
# cut-of'

#Fixes '***' if that makes the last character a punctuation ending mark, otherwise should remove
y='*' 
entries_ending_right=df.loc[df['LAST_CHAR']==y]
entries_ending_right['new_last_char_possible']= entries_ending_right.apply(lambda x:TextCleaning.remove_phrase(x[wa],'***',loc='End')[-1],axis=1)
entries_to_fix=list(entries_ending_right[entries_ending_right['new_last_char_possible'].isin(punctuation)]['PROJECT_ID'])

df[wa]=df.apply(lambda x: TextCleaning.remove_phrase(x[wa],'***','End') 
                if x['PROJECT_ID'] in entries_to_fix else x[wa],axis=1)

df['LAST_CHAR']=df[wa].apply(lambda x: x[-1])

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [9]:
# Using regular expressions to remove "junk" within text body - not necessarily at the start or end

#'Enter the text here tha' ending with 'lines of text.'
expression=re.compile('Enter the text here that.*lines of text')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))

expression=re.compile('PHS .*?Continuation Format Page')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))
expression=re.compile('OMB No .*?Continuation Format Page')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))



df[wa]=df[wa].replace('Project Summary/Abstract','')

"""If it starts with 'one page and must contain',
#This is an NIH thing and there aren't that many of them, but come from 3 different cfda
it will start with "one page and must contain a summary of the proposed activity suitable for dissemination to thepublic. 
It should be a self-contained description of the project and should contain a statement of objectives and methods to be employed.
It should be informative to other persons working in the same or related fields and insofar as possible understandable to a technically liter-ate lay reader. 
This Abstract must not include any proprietary/confidential information.* 
Please click the add attachment button to complete this entry." plus some attachments, which includes tracking number, twice:
following the second trackign number, there is a grant number followed by the actual content" 

At the end of these files, they all end in 'Project Narrative File'(last instance) followed by more attachments, all of which can be discarded
"""
expression1=re.compile('one page and must.*?Tracking Number.*?(Tracking Number)')
expression2=re.compile('Project Narrative File.*')
def fix_abstract(abstract):
    if abstract.startswith('one page and must contain'):
        abstract=re.sub(expression1,'',abstract)
        return re.sub(expression2,'',abstract)
    else:
        return abstract

df[wa]=df[wa].apply(fix_abstract)


df[wa]=df[wa].apply(lambda x: x.lstrip(',;\n\t&-!'))
df=df.loc[df[wa].apply(len)>0]


In [10]:
#starting_exact_phrases to remove
#'This subproject represents an estimate of the percentage of the CTSA funding that isbeing utilized for a broad area of research (AIDS research, pediatric research, orclinical trials).  The Total Cost listed is only an estimate of the amount of CTSAinfrastructure going towards this area of research, not direct funding provided bythe NCRR grant to the subproject or subproject staff.'
#'This subproject is one of many research subprojects utilizing theresources provided by a Center grant funded by NIH/NCRR. The subproject andinvestigator (PI) may have received primary funding from another NIH source,and thus could be represented in other CRISP entries. The institution listed isfor the Center, which is not necessarily the institution for the investigator.'
df[wa]=df[wa].apply(lambda x: x.replace('This subproject represents an estimate of the percentage of the CTSA funding that isbeing utilized for a broad area of research (AIDS research, pediatric research, orclinical trials).  The Total Cost listed is only an estimate of the amount of CTSAinfrastructure going towards this area of research, not direct funding provided bythe NCRR grant to the subproject or subproject staff.',
                                       ''))

expression=re.compile('This subproject is one of many research subprojects.*not necessarily the institution for the investigator.')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))
expression=re.compile('This subproject is one of many research subprojects.*to the subproject or subproject staff.')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))
  

In [11]:
def remove_long_phrase(record):
    """ ignores case to remove multi-word phrases in a particular order, especially those likely to run into other words,
    e.g. Institution university of washingtonPI mary williams. This doesn't work when titles or insititutions have escape characters in them, which is a bummer
    see for example ENHANCING THE USE OF NASA EARTH SCIENCE RESULTS / DATA / AND TECHNOLOGY BY ENGAGING THE FEDERATION OF EARTH SCIENCE INFORMATION PARTNERS COMMUNITIES OF
    PRACTICE IN TARGET AREAS OF INTEREST TO NASA THE FEDERATION OF EARTH SCIENCE INFORMATION PARTNERS (''FED"""
    title=record['PROJECT_TITLE']
    try:
        new_abstract=re.sub(title,'',record[wa],flags=re.IGNORECASE)      
        return re.sub(record['ORGANIZATION_NAME'],'',new_abstract,flags=re.IGNORECASE)   
    except:
        try:
            return re.sub(record['ORGANIZATION_NAME'],'',record[wa],flags=re.IGNORECASE)   
        except:
            return record[wa]

In [12]:
df[wa]=df.apply(lambda x: remove_long_phrase(x),axis=1)

expression=re.compile('Project Summary/Abstract Page.*')

def remove_contact_pd(x):
    """removes clause at end that tends to occur: eg Project Summary/Abstract Page 222Contact PD/PI: Sampson, HughNarrative ("""
    if x.startswith('Contact PD/PI'):
        return re.sub(expression,'',x)
    else:
        return x
df[wa]=df[wa].apply(remove_contact_pd) 

In [13]:
#Drop those that are now length 0 (ie were all punctuation or removable phrases
df.drop(list(df[df[wa].apply(lambda x: len(x)==0)].index),inplace=True)
df['Start Char']=df[wa].apply(lambda x:x[0])
df['LAST_CHAR']=df[wa].apply(lambda x:x[-1])
#df.drop(df[df[wa].apply(len)==0].index[0],axis=0,inplace=True)

In [14]:
# save df 

df.to_pickle("./clean_dataset.pkl")

In [2]:
# save df to CSV

df = pd.read_pickle("./clean_dataset.pkl")

In [3]:
df.to_csv('FRAbstractsSqueakyClean.csv')

In [176]:
# Not running - check with Sam to make sure this is correct


#####################
#Additional expressions we could choose to remove
#Identify abstracts with excessive amounts of other fields to uncover additional bad abstract types
#If we wanted to be on the safe side, some EDA makes me think we could remove anything with more than 3 or 4 of these fields. It's where they start getting wonky.
###################

fields=['Principal Investigator','Program Director','Attachment','Instructions','Lines',
        'Space Provided','Performance Site','Organization','Key Personnel']
all_fields=fields.copy()
all_fields.extend([x.lower() for x in fields])
all_fields.extend([x.upper() for x in fields])
all_fields.extend(['PI','Form','Page','Title','.pdf','.doc'])

def count_up_fields(abstract):
    count=0
    for field in all_fields:
        if field in abstract:
            count+=1
    return count

df['Field Count']=df[wa].apply(count_up_fields)


###########################
#Additional expressions we could remove, but there is a small possibility of some information being lost
##########################

#Issues: 'Close FormNextPrint PageAbout OMB Number']#This is usually ended with "Project summary", 
#so anything between those 2 can be delete, and #ended with a clause starting with 'Close FormProject' and ending in'Narrative File'

#expression1=re.compile('Close FormNext.*?Project Summary')
#expression2=re.compile('Close FormProject.*Narrative File')
def fix_abstract(abstract):
    if abstract.startswith('Close FormNext'):
        abstract=re.sub(expression1,'',abstract)
        return re.sub(expression2,'',abstract)
    else:
        return abstract
#df[wa]=df[wa].apply(fix_abstract)

#If ends in 'Description,', then go to last instance of PERFORMANCE (for Performance SITES), otherwise "KEY PERSONNEL", upper case, and cut all that follows

#expression1=re.compile('PERFORMANCE.*Description,$')
#expression2=re.compile('KEY PERSONNEL.*Description,$')

def apply_expressions(abstract):
    if abstract.endswith('Description,'):
        if re.search(expression1,abstract) != None:
            return re.sub(expression1,'',abstract)
        else:
            return re.sub(expression2,'',abstract)
    else:
        return abstract
    
#df[wa]=df[wa].apply(apply_expressions)

#expression1=re.compile('PERFORMANCE.*Page 3$')
#expression2=re.compile('KEY PERSONNEL.*Page 3,$')

def apply_expressions(abstract):
    if abstract.endswith('Description,'):
        if re.search(expression1,abstract) != None:
            return re.sub(expression1,'',abstract)
        else:
            return re.sub(expression2,'',abstract)
    else:
        return abstract
    
#df[wa]=df[wa].apply(apply_expressions)

